<a href="https://colab.research.google.com/github/mr7495/OCT-classification/blob/main/ROCT_Net_without_Capsules_Kermany_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [2]:
pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 8.8 MB/s 


In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os
import tensorflow as tf

In [4]:
!wget -cO - 'https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/rscbjbr9sj-3.zip' > data.zip

--2022-03-03 23:16:50--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/rscbjbr9sj-3.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.41.184
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.41.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8432836958 (7.9G) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>]   7.85G  21.6MB/s    in 6m 17s  

2022-03-03 23:23:07 (21.4 MB/s) - written to stdout [8432836958/8432836958]



In [5]:
archive = zipfile.ZipFile('data.zip') 
for file in archive.namelist():
     archive.extract(file, '.')

In [6]:
archive = zipfile.ZipFile('ZhangLabData.zip') 
for file in archive.namelist():
     archive.extract(file, 'data')

In [7]:
def find(ch,st):
    indexes=[]
    index1=0
    while(True):
        if ch in st[index1:]:
            index2=st[index1:].index(ch)
            indexes.append(index1+index2)
            index1=index2+index1+1
        
        else:
            break
    return(indexes)


train={'CNV':{},'DME':{},'DRUSEN':{},'NORMAL':{}}
test={'CNV':{},'DME':{},'DRUSEN':{},'NORMAL':{}}

for r,d,f in os.walk('data/CellData/OCT/train'):
    for file in f:
        inds=find('-',file)
        pid=file[inds[0]+1:inds[1]]
        if 'CNV' in r:
            if pid not in train['CNV']:
                train['CNV'][pid]=[]
            train['CNV'][pid].append('{}/{}'.format('CNV',file))
        elif 'DME' in r:
            if pid not in train['DME']:
                train['DME'][pid]=[]
            train['DME'][pid].append('{}/{}'.format('DME',file))
        elif 'DRUSEN' in r:
            if pid not in train['DRUSEN']:
                train['DRUSEN'][pid]=[]
            train['DRUSEN'][pid].append('{}/{}'.format('DRUSEN',file))
        elif 'NORMAL' in r:
            if pid not in train['NORMAL']:
                train['NORMAL'][pid]=[]
            train['NORMAL'][pid].append('{}/{}'.format('NORMAL',file))
            
for r,d,f in os.walk('data/CellData/OCT/test'):
    for file in f:
        inds=find('-',file)
        pid=file[inds[0]+1:inds[1]]
        if 'CNV' in r:
            if pid not in test['CNV']:
                test['CNV'][pid]=[]
            test['CNV'][pid].append('{}/{}'.format('CNV',file))
        elif 'DME' in r:
            if pid not in test['DME']:
                test['DME'][pid]=[]
            test['DME'][pid].append('{}/{}'.format('DME',file))
        elif 'DRUSEN' in r:
            if pid not in test['DRUSEN']:
                test['DRUSEN'][pid]=[]
            test['DRUSEN'][pid].append('{}/{}'.format('DRUSEN',file))
        elif 'NORMAL' in r:
            if pid not in test['NORMAL']:
                test['NORMAL'][pid]=[]
            test['NORMAL'][pid].append('{}/{}'.format('NORMAL',file)) 
    
        
with open('train_cell.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for disease in train:
      if disease=='NORMAL':
          count=0
          for pid in train[disease]:
              csvw.writerow([train[disease][pid][0],disease])
              count+=1
              if count==1000:
                  break
      else:
          for pid in train[disease]:
              csvw.writerow([train[disease][pid][0],disease])
    

with open('test_cell.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for disease in test:
      for pid in test[disease]:
          for item in test[disease][pid]:
              csvw.writerow([item,disease])


In [8]:

train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.1,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1)

test_datagen = keras.preprocessing.image.ImageDataGenerator()

train_df = pd.read_csv("train_cell.csv")
test_df = pd.read_csv("test_cell.csv")

In [9]:
shape=(512, 512)
batch_size=8 #increase when having better GPU
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='data/CellData/OCT/train',
      x_col="filename",
      y_col="class",
      target_size=shape,
      batch_size=batch_size,
      class_mode='categorical',shuffle=True)
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='data/CellData/OCT/test',
        x_col="filename",
        y_col="class",
        target_size=shape,
        batch_size=batch_size,
        class_mode='categorical',shuffle=True)
train_img_num=len(train_generator.filenames)

Found 3213 validated image filenames belonging to 4 classes.
Found 1000 validated image filenames belonging to 4 classes.


In [10]:
!git clone https://github.com/mhrahimzadeh1374/automl

Cloning into 'automl'...
remote: Enumerating objects: 3922, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 3922 (delta 192), reused 215 (delta 163), pack-reused 3613
Receiving objects: 100% (3922/3922), 23.18 MiB | 20.36 MiB/s, done.
Resolving deltas: 100% (2944/2944), done.


In [11]:
cd automl/efficientnetv2

/content/automl/efficientnetv2


In [12]:
from effnetv2_model import get_model

In [13]:
cd ..

/content/automl


In [14]:
cd ..

/content


In [ ]:
name="ROCT-Net_without Capsules_Kermany Dataset"
!mkdir "models"

# ROCT-Net model withoud Capsule network
keras.backend.clear_session()
input_tensor=keras.Input(shape=(shape[0],shape[1],3))
base_model1=get_model('efficientnetv2-b0', include_top=False, pretrained=True)(input_tensor)
base_model2=keras.applications.Xception(input_tensor=input_tensor,weights='imagenet',include_top=False)(input_tensor)

concatenated=keras.layers.concatenate([base_model1,base_model2]) 

avg=keras.layers.AveragePooling2D(3,padding='valid')(concatenated)
depthw=keras.layers.DepthwiseConv2D(5,
                                      depthwise_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                                      bias_initializer=keras.initializers.Zeros(),depthwise_constraint=keras.constraints.NonNeg())(avg)
flat=keras.layers.Flatten()(depthw)
dp=keras.layers.Dropout(0.2)(flat)
preds=keras.layers.Dense(4,activation='softmax',
                          kernel_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                          bias_initializer=keras.initializers.Zeros(),)(dp)
model=keras.Model(inputs=input_tensor, outputs=preds)  

##################################
for layer in model.layers:
  layer.trainable = True
model.summary()
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%name
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max',save_weights_only=True) #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]


lr_schedule =keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.045,
    decay_steps=2*int(train_img_num/batch_size),
    decay_rate=0.94,
    staircase=True)
optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

hist=model.fit_generator(train_generator, epochs=40,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training
with open('{}-results.csv'.format(name), mode='w',newline='') as csv_file: #write evaluation metrics
  csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  for key in hist.history:
    data=[key]
    data.extend(hist.history[key])
    csv_writer.writerow(data)